Note...would be good to do a test for the version of Chrome and provide an error if the Chromedriver version doesn't match that includes the link to download the chromedriver and also directions on where to put it (path)

# Imports

## Selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import os
import time
from datetime import datetime
import pyotp # for 2 factor

In [2]:
def rk_studio_login(username, password, otp_code):
    # go to site
    driver.get('https://rkstudio.careevolution.com/inv')
    
    # enter username and password, then submit
    driver.find_element_by_id('Username').send_keys(username)
    driver.find_element_by_id ('Password').send_keys(password)
    driver.find_element_by_id('login-button').click()
    
    # get 2 factor authorization code, then submit
    totp = pyotp.TOTP(otp_code)
    two_factor = totp.now()
    driver.find_element_by_id ('Code').send_keys(two_factor)
    driver.find_element_by_id('provide-one-time-password-submit').click()
    
    # navigate to project
    try:
        # make sure page is loaded first, then pull link
        link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "DJW Thesis"))
        )
        link.click()
        
    except:
        print("Something went wrong going to Thesis project...")
        driver.quit()

    # navigate to downloads
    try:
        # make sure page is loaded first, then pull link
        link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.LINK_TEXT, "Export History"))
        )
        link.click()
        
    except:
        print("Something went wrong going to Export History...")
        driver.quit()
    
    #-----------------------------------#
    # find date of last downloaded file #
    #-----------------------------------#
    current_files = []

    # find all files in raw_data/testing directory
    for root, dirs, files in os.walk(download_path):
        for filename in files:
            if filename.endswith('.zip'):
                filename = filename.split('-')[1].split('.')[0]
                current_files.append(filename)

    # put files in order by date
    current_files.sort()

    # get the last element in sorted list and convert to datetime
    last_download = datetime.strptime(current_files[-1], '%Y%m%d')
    
    #----------------------------------#
    # Download new data from RK Studio #
    #----------------------------------#
    
    # get buttons of files (to click)
    try:
        # make sure page loads first...
        buttons = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//button[@class='editor-controls-button' and @title='Download']"))
        )
        # buttons = driver.find_elements_by_xpath("//button[@class='editor-controls-button' and @title='Download']")
    
        # get dates of files
        dates = driver.find_elements_by_xpath("//*[contains(text(),'Incremental')]")
    except:
        print("Something went wrong getting buttons...")
    
    # create list of dates to check against
    file_dates = []

    for date in dates:
        # check if empty
        if date.text:
            date_text = date.text.split(',')[1].split('to ')[-1] + ' ' +  date.text.split(',')[2].split(' ')[1]
            date_dt = datetime.strptime(date_text, '%b %d %Y')
            file_dates.append(date_dt)

    # download files more recent than `last_download`
    for i in range(len(file_dates)):
        if file_dates[i] > last_download:
            buttons[i].click()
    
    # give it time to download
    time.sleep(30)

    # output time completed
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("Download completed: ", dt_string)


    # end session 
    driver.quit()

In [3]:
# Chromedriver path
PATH = "/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_3_data_analysis/resources/chromedriver"

# Login credentials
user_auth = {'Username': 'djw',
             'Password': 'nFs3NdlXTKat65Auv!4Z!i#',
             'OTP code': 'dp2dL56du3nmoL2prv6x7ryfyowu5pLo'
            }

In [4]:
download_path = "/Users/djw/Documents/pCloud_synced/Academics/Projects/2020_thesis/thesis_experiments/3_experiments/3_3_experience_sampling/3_3_1_raw_data/testing"

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_path}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(PATH, options=chrome_options)

# login
rk_studio_login(user_auth['Username'], user_auth['Password'], user_auth['OTP code'])

## 2 Factor Auth

[PyOTP](https://pyauth.github.io/pyotp/) is a Python library for generating and verifying one-time passwords. It can be used to implement two-factor (2FA) or multi-factor (MFA) authentication methods in web applications and in other systems that require users to log in.